# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 60 new papers today
          20 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/19 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2211.09136


extracting tarball to tmp_2211.09136...

 done.
Retrieving document from  https://arxiv.org/e-print/2211.09149


extracting tarball to tmp_2211.09149...

...

/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Found documentclass in tmp_2211.09617/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'journaldef.tex' from 'tmp_2211.09617/journaldef.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'journaldef.tex' from 'tmp_2211.09617/journaldef.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


extracting tarball to tmp_2211.09628...

 done.
Retrieving document from  https://arxiv.org/e-print/2211.09635


extracting tarball to tmp_2211.09635...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.09217-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.09217) | **Stellar Properties for a Comprehensive Collection of Star Forming  Regions in the SDSS APOGEE-2 Survey**  |
|| Carlos G. Román-Zúñiga, et al. -- incl., <mark>Jesús Hernández</mark>, <mark>D. A. García-Hernández</mark> |
|*Appeared on*| *2022-11-18*|
|*Comments*| *6 figures, 4 tables*|
|**Abstract**| The Sloan Digital Sky Survey IV (SDSS-IV) APOGEE-2 primary science goal was to observe red giant stars throughout the Galaxy to study its dynamics, morphology, and chemical evolution. The APOGEE instrument, a high-resolution 300 fiber H-band (1.55-1.71 micron) spectrograph, is also ideal to study other stellar populations in the Galaxy, among which are a number of star forming regions and young open clusters. We present the results of the determination of six stellar properties ($T_{eff}$, $\log{g}$, [Fe/H], $L/L_\odot$, $M/M_\odot$, and ages) for a sample that is composed of 3360 young stars, of sub-solar to super-solar types, in sixteen Galactic star formation and young open cluster regions. Those sources were selected by using a clustering method that removes most of the field contamination. Samples were also refined by removing targets affected by various systematic effects of the parameter determination. The final samples are presented in a comprehensive catalog that includes all six estimated parameters. This overview study also includes parameter spatial distribution maps for all regions and Hertzprung-Russell ($L/L_\odot$ vs. $T_{eff}$) diagrams. This study serves as a guide for detailed studies on individual regions, and paves the way for the future studies on the global properties of stars in the pre-main sequence phase of stellar evolution using more robust samples. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.09309-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.09309) | **Charging of free-falling test masses in orbit due to cosmic rays:  results from LISA Pathfinder**  |
|| LISA Pathfinder Collaboration, et al. -- incl., <mark>I. Harrison</mark>, <mark>G. Heinzel</mark>, <mark>M. Hewitson</mark>, <mark>J. Ramos-Castro</mark> |
|*Appeared on*| *2022-11-18*|
|*Comments*| *16 pates, 9 figures*|
|**Abstract**| A comprehensive summary of the measurements made to characterize test mass charging due to the space environment during the LISA Pathfinder mission is presented. Measurements of the residual charge of the test mass after release by the grabbing and positioning mechanism, show that the initial charge of the test masses was negative after all releases, leaving the test mass with a potential in the range $-12$ mV to $-512$ mV. Variations in the neutral test mass charging rate between $21.7$ e s$^{-1}$ and $30.7$ e s$^{-1}$ were observed over the course of the 17-month science operations produced by cosmic ray flux changes including a Forbush decrease associated with a small solar energetic particle event. A dependence of the cosmic ray charging rate on the test mass potential between $-30.2$ e s$^{-1}$ V$^{-1}$ and $-40.3$ e s$^{-1}$ V$^{-1}$ was observed and this is attributed to a contribution to charging from low-energy electrons emitted from the gold surfaces of the gravitational reference sensor. Data from the on-board particle detector show a reliable correlation with the charging rate and with other environmental monitors of the cosmic ray flux. This correlation is exploited to extrapolate test mass charging rates to a 20-year period giving useful insight into the expected range of charging rate that may be observed in the LISA mission. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.09492-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.09492) | **Photometric redshift estimation of galaxies in the DESI Legacy Imaging  Surveys**  |
|| <mark>Changhua Li</mark>, et al. -- incl., <mark>Yanxia Zhang</mark>, <mark>Yongheng Zhao</mark>, <mark>Jing-Yi Zhang</mark>, <mark>Shanshan Li</mark>, <mark>Linying Mi</mark>, <mark>Boliang He</mark> |
|*Appeared on*| *2022-11-18*|
|*Comments*| *Accepted for publication in MNRAS. 14 pages, 9 figures, 11 tables*|
|**Abstract**| The accurate estimation of photometric redshifts plays a crucial role in accomplishing science objectives of the large survey projects. The template-fitting and machine learning are the two main types of methods applied currently. Based on the training set obtained by cross-correlating the DESI Legacy Imaging Surveys DR9 galaxy catalogue and SDSS DR16 galaxy catalogue, the two kinds of methods are used and optimized, such as EAZY for template-fitting approach and CATBOOST for machine learning. Then the created models are tested by the cross-matched samples of the DESI Legacy Imaging SurveysDR9 galaxy catalogue with LAMOST DR7, GAMA DR3 and WiggleZ galaxy catalogues. Moreover three machine learning methods (CATBOOST, Multi-Layer Perceptron and Random Forest) are compared, CATBOOST shows its superiority for our case. By feature selection and optimization of model parameters, CATBOOST can obtain higher accuracy with optical and infrared photometric information, the best performance ($MSE=0.0032$, $\sigma_{NMAD}=0.0156$ and $O=0.88$ per cent) with $g \le 24.0$, $r \le 23.4$ and $z \le 22.5$ is achieved. But EAZY can provide more accurate photometric redshift estimation for high redshift galaxies, especially beyond the redhisft range of training sample. Finally, we finish the redshift estimation of all DESI DR9 galaxies with CATBOOST and EAZY, which will contribute to the further study of galaxies and their properties. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.09548-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.09548) | **Asgard/NOTT: L-band nulling interferometry at the VLTI I. Simulating the  expected high-contrast performance**  |
|| Romain Laugier, et al. -- incl., <mark>Simon Gross</mark> |
|*Appeared on*| *2022-11-18*|
|*Comments*| **|
|**Abstract**| Context: NOTT (formerly Hi-5) is a new high-contrast L' band (3.5-4.0 \textmu m) beam combiner for the VLTI with the ambitious goal to be sensitive to young giant exoplanets down to 5 mas separation around nearby stars. The performance of nulling interferometers in these wavelengths is affected both by fundamental noise from the background and by the contributions of instrumental noises. This motivates the development of end-to-end simulations to optimize these instruments. Aims: To enable the performance evaluation and inform the design of such instruments on the current and future infrastructures, taking into account the different sources of noise, and their correlation. Methods: SCIFYsim is an end-to-end simulator for single mode filtered beam combiners, with an emphasis on nulling interferometers. It is used to compute a covariance matrix of the errors. Statistical detection tests based on likelihood ratios are then used to compute compound detection limits for the instrument. Results: With the current assumptions on the performance of the wavefront correction systems, the errors are dominated by correlated instrumental errors down to stars of magnitude 6-7 in the L band, beyond which thermal background from the telescopes and relay system becomes dominant. Conclusions: SCIFYsim is suited to anticipate some of the challenges of design, tuning, operation and signal processing for integrated optics beam combiners. The detection limits found for this early version of NOTT simulation with the unit telescopes are compatible with detections at contrasts up to $10^5$ in the L band at separations of 5 to 80 mas around bright stars. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.09583-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.09583) | **Linking the Internal Properties of Infant Globular Clusters to their  Formation Environments**  |
|| Frederika Phipps, Sadegh Khochfar, <mark>Anna Lisa Varri</mark>, Claudio Dalla Vecchia |
|*Appeared on*| *2022-11-18*|
|*Comments*| *17 pages, 12 Figures, Accepted by MNRAS*|
|**Abstract**| We investigate the formation of infant globular cluster (GC) candidates in high-resolution cosmological simulations from the First Billion Years (FiBY) project. By analysing the evolution of the systems in the energy and angular momentum plane, we identify the redshift at which the infant GCs first became gravitationally bound, and we find evidence of radial infall of their gaseous and stellar components. The collapse appears to be driven by internal self-gravity, however, the initial trigger is sourced from the external environment. The phase space behaviour of the infant GCs also allows us to identify some characteristic groupings of objects. Such a classification based on internal properties appears to be reflected in the formation environment: GC candidates that belong to the same class are found in host galaxies of similar morphology, with the majority of the infant GCs located in clumpy, irregular proto-galaxies. Finally, through the inspection of two GC candidates that contain only stars by z = 6, we find that supernova feedback is the main physical mechanism behind their dearth of gas and that the systems subsequently respond with an approximately adiabatic expansion. Such infant GC candidates already resemble the GCs we currently observe in the local Universe. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.09617-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.09617) | **Detecting deviations from Gaussianity in high-redshift CMB lensing maps**  |
|| <mark>Zhuoqi Zhang</mark>, Yuuki Omori, Chihway Chang |
|*Appeared on*| *2022-11-18*|
|*Comments*| *9 pages, 9 figures. Comments welcome*|
|**Abstract**| While the probability density function (PDF) of the cosmic microwave background (CMB) convergence field approximately follows a Gaussian distribution, small contributions from structures at low redshifts make the overall distribution slightly non-Gaussian. Some of this late-time component can be modelled using the distribution of galaxies and subtracted off from the original CMB lensing map to produce a map of matter distribution at high redshifts. Using this high-redshift mass map, we are able to directly study the early phases of structure formation and look for deviations from our standard model. In this work, we forecast the detectability of signatures of non-Gaussianity due to nonlinear structure formation at $z>1.2$. Although we find that detecting such signatures using ongoing surveys will be challenging, we forecast that future experiments such as the CMB-S4 will be able to make detections of $\sim$ 7$\sigma$. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.09628-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.09628) | **A Detailed Look at the Most Obscured Galactic Nuclei in the Mid-Infrared**  |
|| F. R. Donnan, et al. -- incl., <mark>A. Alonso-Herrero</mark>, <mark>A. Hernán-Caballero</mark> |
|*Appeared on*| *2022-11-18*|
|*Comments*| *Accepted for publication in A&A. 23 pages, 15 figures, 3 tables*|
|**Abstract**| Context. Compact Obscured Nuclei (CONs) are an extreme phase of galaxy evolution where rapid supermassive black hole growth and$/$or compact star-forming activity is completely obscured by gas and dust. Aims. We investigate the properties of CONs in the mid-infrared and explore techniques aimed at identifying these objects such as through the equivalent width (EW) ratios of their Polycyclic Aromatic Hydrocarbon (PAH) features. Methods. We model Spitzer spectra by decomposing the continua into nuclear and star-forming components from which we then measure the nuclear optical depth, $\tau_N$, of the $9.8 \mu$m silicate absorption feature. We also use Spitzer spectral maps to investigate how PAH EW ratios vary with aperture size for objects hosting CONs. Results. We find that the nuclear optical depth, $\tau_N$, strongly correlates with the HCN-vib emission line in the millimetre for CONs with a Pearson correlation coefficient of 0.91. We find the PAH EW ratios technique to be effective at selecting CONs and robust against highly inclined galaxies where strong dust lanes may mimic a CON like spectrum by producing a high $\tau_N$. Our analysis of the Spitzer spectral maps showed that the efficacy of the PAH EW ratios to isolate CONs is reduced when there is a strong star-forming component from the host galaxy. In addition, we find that the use of the inferred nuclear optical depth is a reliable method to identify CONs in $36^{+8}_{-7}\%$ of ULIRGs and $17^{+3}_{-3}\%$ of LIRGs, consistent with previous work. Conclusions. We confirm mid-IR spectra to be a powerful diagnostic of CONs where the increased sensitivity of JWST will allow identification of CONs at cosmic noon revealing this extreme but hidden phase of galaxy evolution. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.09635-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.09635) | **The role of the magnetic fields in GRB outflows**  |
|| N. Jordana-Mitjans, et al. -- incl., <mark>R. J. Smith</mark>, <mark>V. Lipunov</mark> |
|*Appeared on*| *2022-11-18*|
|*Comments*| *To appear in the Proceedings of the 16th Marcel Grossmann Meeting (July 5-10, 2021)*|
|**Abstract**| Gamma-ray bursts (GRBs) are bright extragalactic flashes of gamma-ray radiation and briefly the most energetic explosions in the Universe. Their catastrophic origin (the merger of compact objects or the collapse of massive stars) drives the formation of a newborn compact remnant (black hole or magnetar) that powers two highly relativistic jets. To distinguish between magnetized and baryonic jet models and ultimately determine the power source for these energetic explosions, our team studies the polarization of the light during the first minutes after the explosion (using novel instruments on fully autonomous telescopes around the globe) to directly probe the magnetic field properties in these extragalactic jets. This technology allowed the detection of highly polarized optical light in GRB 120308A and confirmed the presence of mildly magnetized jets with large-scale primordial magnetic fields in a reduced sample of GRBs (e.g. GRB 090102, GRB 110205A, GRB 101112A, GRB 160625B). Here we discuss the observations of the most energetic and first GRB detected at very high TeV energies, GRB 190114C, which opens a new frontier in GRB magnetic field studies suggesting that some jets can be launched highly magnetized and that the collapse and destruction of these magnetic fields at very early times may have powered the explosion itself. Additionally, our most recent polarimetric observations of the jet of GRB 141220A indicate that, when the jetted ejected material is decelerated by the surrounding environment, the magnetic field amplification mechanisms at the front shock (needed to generate the observed synchrotron emission) produce small magnetic domains. These measurements validate theoretical expectations and contrast with previous observations that suggest large magnetic domains in collisionless shocks (i.e. GRB 091208B). |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2211.09217.md
    + _build/html/tmp_2211.09217/././fig06.png
    + _build/html/tmp_2211.09217/././fig01.png
    + _build/html/tmp_2211.09217/././fig02.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$</div>



<div id="title">

# Based on SDSS Data Releases 16 and 17

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2211.09217-b31b1b.svg)](https://arxiv.org/abs/2211.09217)<mark>Appeared on: 2022-11-18</mark> - _6 figures, 4 tables_

</div>
<div id="authors">

Carlos G. Rom\'an-Z\'u\~niga, et al. -- incl., <mark>Jes\'us Hern\'andez</mark>, <mark>D. A. Garc\'ia-Hern\'andez</mark>

</div>
<div id="abstract">

**Abstract:** The Sloan Digital Sky Survey IV (SDSS-IV) APOGEE-2 primary science goal was to observe red giant stars throughout the Galaxy to study its dynamics, morphology, and chemical evolution. The APOGEE instrument, a high- resolution 300 fiber H-band (1.55-1.71 micron) spectrograph, is also ideal to study other stellar populations in the Galaxy, among which are a number of star forming regions and young open clusters. We present the results of the determination of six stellar properties (\teff,\logg,[Fe/H],$\mathrm{L/L_\odot, M/M_\odot, and  age}$) for a sample that is composed of 3360  young stars, of sub-solar to super-solar types, in sixteen Galactic star formation and young open cluster regions. Those sources were selected by using a clustering method that removes most of the field contamination. Samples were also refined by removing targets affected by various systematic effects of the parameter determination. The final samples are presented in a comprehensive catalog that includes all six estimated parameters. This overview study also includes parameter spatial distribution maps for all regions and Hertzprung-Russell ($\mathrm{\log{L/L_\odot}}$versus\teff) diagrams. This study serves as a guide for detailed studies on individual regions, and paves the way for the future studies on the global properties of stars in the pre-main sequence phase of stellar evolution using more robust samples.

</div>

<div id="div_fig1">

<img src="tmp_2211.09217/././fig06.png" alt="Fig1" width="100%"/>

**Figure 1. -** Effective temperature vs. Gaia EDR3 colors for the APOGEE Pleiades sample. The symbol colors represent age estimates from \texttt{MassAge}. The solid line is the empirical sequence from \citet{Esplin18}. \label{fig:pleidisc} (*fig:pleidisc*)

</div>
<div id="div_fig2">

<img src="tmp_2211.09217/././fig01.png" alt="Fig2" width="100%"/>

**Figure 2. -** A map, in Galactic coordinates, showing the locations of all the fields listed in Table \ref{tab:yso_sample}, along with labels identifying the eighteen main regions reported in this study. Each circular field is projected with a realistic radius of 1.5 or 1.0 deg for regions observed with the North or South APOGEE instruments, respectively. The colors of the fields refer to the field-subprograms described in Section \ref{sec:observations}. \label{fig:bigmap} (*fig:bigmap*)

</div>
<div id="div_fig3">

<img src="tmp_2211.09217/././fig02.png" alt="Fig3" width="100%"/>

**Figure 3. -** APOGEE-2 Cygnus-X Complex Fields are shown, delimited with circles; red circles indicate the main region fields, and the dotted blue circles indicate "auxiliary" fields (see Appendix \ref{App:sub-programs}) also searched for candidate members. Color dot symbols indicate the positions of all scientific targets.
The grayscale image in the background shows a 12 $\mu$m dust emission map from the
WISE WSSA Atlas \citep{WSSA14}. The colorbar indicates values of the signal-to-noise ratio after visit combination. The component figures (51) for all the regions studied in this paper, are available online in the Figure Set \ref{FS1}, which also include maps highlighting H mag brightness and Number of Visits.
\label{fig:posmap} (*fig:posmap*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

94  publications in the last 7 days.
	 _build/html/2211.09217.md
	 _build/html/2211.08493.md
	 _build/html/2211.08487.md
	 _build/html/2211.08294.md
	 _build/html/2211.07667.md
	 _build/html/2211.07659.md
	 _build/html/2211.06454.md
	 _build/html/2211.06205.md
	 _build/html/2211.06194.md
	 _build/html/2211.05022.md
	 _build/html/2211.04048.md
	 _build/html/2211.03641.md
	 _build/html/2211.02502.md
	 _build/html/2211.01474.md
	 _build/html/2211.00667.md
	 _build/html/2211.00657.md
	 _build/html/2211.00087.md
	 _build/html/2211.00036.md
	 _build/html/2211.00020.md
	 _build/html/2211.00010.md
	 _build/html/2210.17434.md
	 _build/html/2210.12412.md
	 _build/html/2210.11926.md
	 _build/html/2210.11551.md
	 _build/html/2210.11103.md
	 _build/html/2210.10707.md
	 _build/html/2210.09930.md
	 _build/html/2210.08996.md
	 _build/html/2210.08517.md
	 _build/html/2210.08510.md
	 _build/html/2210.08078.md
	 _build/html/2210.06504.md
	 _build/html/2210.05695.md
	 _build/html/2210.05595.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers